In [ ]:
import os
import numpy as np
import gc
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve

2025-03-08 16:33:09.026866: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 16:33:09.165237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741426389.221766  372474 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741426389.236559  372474 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 16:33:09.362672: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
N_MELS = 128

In [ ]:
X = np.load('../SavedFeatures/X_mel_spec.npy')
y = np.load('../SavedFeatures/y_mel_spec.npy')

X_train, X_val, y_train, y_val= train_test_split(X, y, test_size = 0.2, random_state = 42)

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

del X, y  
gc.collect()

In [17]:
def cus_CNN(input_shape, num_classes):
    model_input = Input(shape=input_shape) 
    x = Conv2D(32, kernel_size=(3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x) 
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.5)(x)  
    
    x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.4)(x)  

    x = Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)  
    x = Flatten()(x)

    x = Dropout(0.2)(x)  
    output = Dense(num_classes, activation='softmax')(x) 
    return Model(inputs=model_input, outputs=output)

In [ ]:
cnnmodel = cus_CNN(input_shape=(N_MELS, 109, 1), num_classes=2)
cnnmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [19]:
tf.keras.backend.clear_session()

In [20]:
cnnmodel.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.6835 - loss: 0.7762 - val_accuracy: 0.8308 - val_loss: 0.5083
Epoch 2/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7979 - loss: 0.4826 - val_accuracy: 0.8347 - val_loss: 0.4473
Epoch 3/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8430 - loss: 0.3879 - val_accuracy: 0.5646 - val_loss: 0.6787
Epoch 4/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8713 - loss: 0.3289 - val_accuracy: 0.8524 - val_loss: 0.3596
Epoch 5/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8843 - loss: 0.2897 - val_accuracy: 0.9105 - val_loss: 0.2762
Epoch 6/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9033 - loss: 0.2495 - val_accuracy: 0.9062 - val_loss: 0.2342
Epoch 7/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9072 - loss: 0.2333 - val_accuracy: 0.8901 - val_loss: 0.2752
Epoch 8/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9195 - loss: 0.2140 - val_accuracy: 0.

In [ ]:
test_loss, test_accuracy = cnnmodel.evaluate(X_val, y_val, verbose=0)
y_pred = cnnmodel.predict(X_val)
y_pred_classes = y_pred.argmax(axis=1)
y_true_classes = y_val.argmax(axis=1)

f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
print("===================CNN - MS===================")
print(f"Test Accuracy: {test_accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")


# Tính EER
eers = []
for i in range(y_pred.shape[1]):
    y_true_binary = y_val[:, i]
    y_pred_prob = y_pred[:, i]
    fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_prob)
    fnr = 1 - tpr
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]
    eers.append(eer)
    print(f"Class {i}: EER = {eer:.5f} at threshold {eer_threshold:.5f}")

mean_eer = np.mean(eers)
print(f"EER: {mean_eer:.5f}")

80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
===================CNN - MS===================
Test Accuracy: 0.91991
F1-Score: 0.91803
Class 0: EER = 0.05382 at threshold 0.99304
Class 1: EER = 0.05377 at threshold 0.00699
EER: 0.05379


In [22]:
# Save model as H5 File
cnnmodel.save("MS_cus_CNN.h5")

In [ ]:

del cnnmodel, X_train, X_val, y_train, y_val
gc.collect()


1043